In [1]:
import pandas as pd
from time import sleep
from urllib.error import HTTPError

In [2]:
some_ret = pd.read_html("https://www.fda.gov/medical-devices/software-medical-device-samd/artificial-intelligence-and-machine-learning-aiml-enabled-medical-devices")
dataframe = some_ret[0]

In [3]:
dataframe.head()

,Date of Final Decision,Submission Number,Device,Company,Panel (lead),Primary Product Code
0,09/27/2024,K240369,CAC (gated) Algorithm,BunkerHill Health,Radiology,JAK
1,09/27/2024,K241480,JBS-LVO,"JLK, Inc.",Radiology,QAS
2,09/27/2024,K240993,encevis (2.1),AIT Austrian Institute of Technology GmbH,Neurology,OMB
3,09/24/2024,K242292,uAI Easy Triage ICH,"Shanghai United Imaging Intelligence Co., Ltd.",Radiology,QAS
4,09/23/2024,K240013,EchoGo Heart Failure (2.0),Ultromics Limited,Cardiovascular,QUO


In [2]:
csvFileName = "dataframe.csv"

In [ ]:
print(dataframe.__len__())
dataframe.to_csv(csvFileName, index=False)

1016


In [3]:
dataframe = pd.read_csv(csvFileName, index_col=None)
# read_DF.head()
dataframe.__len__()

1016

In [5]:
dataframe.columns

Index(['Date of Final Decision', 'Submission Number', 'Device', 'Company',
       'Panel (lead)', 'Primary Product Code'],
      dtype='object')

In [4]:
dataframe["Panel (lead)"].unique()

array(['Radiology', 'Neurology', 'Cardiovascular', 'Anesthesiology',
       'General and Plastic Surgery', 'Orthopedic',
       'Gastroenterology-Urology', 'Dental', 'Ophthalmic', 'Hematology',
       'General Hospital', 'Pathology', 'Clinical Chemistry',
       'Microbiology', 'Obstetrics and Gynecology', 'Clinical Tcxicology',
       'Immunology'], dtype=object)

In [5]:
dataframe["Submission Number"]

0       K240369
1       K241480
2       K240993
3       K242292
4       K240013
         ...   
1011    K033840
1012    K003301
1013    P970058
1014    K955841
1015    P940029
Name: Submission Number, Length: 1016, dtype: object

In [4]:
required_data = dataframe.loc[dataframe["Panel (lead)"] == "Radiology"]
required_data.reset_index(inplace=True)

In [7]:
required_data["Submission Number"]

0      K240369
1      K241480
2      K242292
3      K240036
4      K241582
        ...   
772    K120161
773    K111776
774    K082483
775    K080762
776    P970058
Name: Submission Number, Length: 777, dtype: object

In [4]:
all_lengths = {}
pdf_urls = []
sleep_time_recommended = 25 # in seconds
devices_already_done = 10
devices_needed_currently = 10
file_with_URLs = "pdf_urls.txt"

In [16]:
for i in range(devices_already_done, devices_already_done+devices_needed_currently):
    try:
        sleep(sleep_time_recommended)
        device_ID = required_data["Submission Number"][i]
        specific_device_FDA_URL = f"https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfPMN/pmn.cfm?ID={device_ID}"
        some_ret = pd.read_html(specific_device_FDA_URL) # Pandas DataFrame object
        # For single digit years its just single digit -> 4 not 04
        year_submitted = str(int(some_ret[7].loc[some_ret[7][0] == "Date Received"][1].values[0][-2:]))
        pdf_URL = f"https://www.accessdata.fda.gov/cdrh_docs/pdf{year_submitted}/{device_ID}.pdf"
        pdf_urls.append(pdf_URL)
        len_considered = some_ret[7].__len__()
        if len_considered not in all_lengths:
            all_lengths[len_considered] = 0
        all_lengths[len_considered] += 1
        print(f"Finished device {i+1}...")
        if not ((i+1)%5):
            with open(file_with_URLs, "a") as f:
                for i in range(len(pdf_urls)):
                    f.write(pdf_urls[i]+"\n")
            pdf_urls = []
    except HTTPError:
        print(HTTPError)
print(all_lengths)
with open(file_with_URLs, "a") as f:
    for i in range(len(pdf_urls)):
        f.write(pdf_urls[i]+"\n")

Finished device 11...
Finished device 12...
Finished device 13...
Finished device 14...
Finished device 15...
Finished device 16...
Finished device 17...
Finished device 18...
Finished device 19...
Finished device 20...
{20: 5, 19: 5}


Some devices have 19 whereas some have 20 fields. 2 devices made before Y2K have 16 to 17 fields with updates and amendments to original statements.

The difference arises in the addition of "Subsequent Product Code" in addition to the "Classification Product Code"

In [13]:
some_ret[7] # Some have 20, some have 19

,0,1
0,Device Classification Name,Automated Radiological Image Processing Software
1,510(k) Number,K240036
2,Device Name,PIUR tUS Infinity
3,Applicant,PIUR Imaging GmbH Hamburgerstr. 11/Top 7 Vie...
4,Applicant Contact,Milos Stojanovic
5,Correspondent,"JW Regulatory Consulting LLC 406 Wacouta St, ..."
6,Correspondent Contact,Jennifer Willner
7,Regulation Number,892.2050
8,Classification Product Code,QIH
9,Date Received,01/05/2024


In [2]:
import requests, io, pytesseract
import fitz as fz #PyMuPdf
# from PyPDF2 import PdfReader
from PIL import Image
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [5]:
with open(file_with_URLs) as url_file:
    all_lines = url_file.readlines()

In [ ]:
current_url = all_lines[0][:-1] # Skips the "\n" portion at the end of each line
# url_to_read_from = "https://www.accessdata.fda.gov/cdrh_docs/pdf8/K080762.pdf"
request = requests.get(current_url) # Only here the request is made
filestream = io.BytesIO(request.content)

In [7]:
doc = fz.open(stream=filestream, filetype="pdf")
ocr_text = ""
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    pix = page.get_pixmap(matrix=fz.Matrix(2,2))
    img = Image.open(io.BytesIO(pix.tobytes()))

    text = pytesseract.image_to_string(img)
    ocr_text += f"\n--- Page {page_num + 1} (OCR) ---\n"
    ocr_text += text
doc.close()

In [11]:
print(ocr_text) # New doc - 2024 - 


--- Page 1 (OCR) ---
ADMINISTRATION

Lg Py U.S. FOOD & DRUG

September 27, 2024

BunkerHill Health

% Nitya Narayanan

Director of Regulatory Affairs
436 Bryant Street

SAN FRANCISCO, CA 94107

Re: K240369
Trade/Device Name: CAC (gated) Algorithm
Regulation Number: 21 CFR 892.1750
Regulation Name: Computed Tomography X-Ray System
Regulatory Class: Class II
Product Code: JAK
Dated: August 23, 2024
Received: August 26, 2024

Dear Nitya Narayanan:

We have reviewed your section 510(k) premarket notification of intent to market the device referenced above
and have determined the device is substantially equivalent (for the indications for use stated in the enclosure)
to legally marketed predicate devices marketed in interstate commerce prior to May 28, 1976, the enactment
date of the Medical Device Amendments, or to devices that have been reclassified in accordance with the
provisions of the Federal Food, Drug, and Cosmetic Act (the Act) that do not require approval of a premarket
approval

In [ ]:
print(ocr_text) # Older doc - 2008 - 14.4s


--- Page 1 (OCR) ---
Lov7162

EXHIBIT #1

510(k) SUMMARY

This summary of 510(k) safety and effectiveness information is being submitted in
accordance with the requirements of SMDA 1990 and 21 CFR §807.92.

The assigned 510(k) number is:

1.

MAY 1
Submitter's Identification: 5 2008
Imaging Biometrics, LLC
1035 Katherine Drive
Elm Grove, WI 53122
Contact Person
Michael Schmainda
Phone: (262) 385-1796
Email: mike@imagingbiometrics.com
Date Summary Prepared: 3/4/2008
Name of the Device:
Trade Name: IB Neuro™ v1.0
Classification Name: MR Diagnostic Device Accessory: 21 CFR 892.1000
Classification: Class II
Product Code: LNH
Classification Name: Picture Archiving and Communication Systems
21 CFR 892.2050
Classification: Class II
Product Code: LLZ

Common or Usual Name:
IB Neuro™ 1.0 / PACS device
Predicate Device Information:

IB Neuro™ is substantially equivalent to the following legally marketed devices
that are currently cleared by the FDA:

--- Page 2 (OCR) ---
 

510(k) Trade Name M

In [13]:
import ollama
model = "llama3.2:1b"

In [1]:
details_required = ["AI Product name", "AI Company name", "Sub-specialty", "Imaging Modality", "Intended Purpose", "Clinical use", "Product output", "CE Classification", "FDA Classification", "Manufacturer"]

In [ ]:
prompt = "Given below are the contents from an FDA regulatory document. I require the following details from the document: " + ", ".join(details_required) + ". Give me these information in a JSON format only."
# prompt = "Given above are the contents from an FDA regulatory document. Give me the following 10 details only from the document: AI Product name, AI Company name, Sub-specialty, Imaging Modality, Intended Purpose, Clinical use, Product output, CE Classification, FDA Classification, Manufacturer. Give me no additional details"
document_contents = ocr_text

In [15]:
response = ollama.chat(model=model, messages=[
    {
        "role": "user",
        "content": f"--- Document Contents: ---\n{document_contents}\n\n{prompt}"
    }
])
print(response.message.content)

Here are the 10 details:

1. **AI Product Name**: BunkerHill Health Inc.
2. **AI Company Name**: BunkerHill
3. **Sub-specialty**: None specified
4. **Imaging Modality**: Computed Tomography (CT)
5. **Intended Purpose**: Estimating presence and quantity of coronary artery calcium (CAC) for patients aged 30 years and above
6. **Clinical use**: Routine care
7. **Product output**: DICOM image with Agatston score and other relevant information
8. **CE Classification**: None specified, but it is said to be "Same as predicate devices"
9. **FDA Classification**: Not explicitly mentioned, but implied to be Class III (combination of safety and effectiveness)
10. **Manufacturer**: BunkerHill Health Inc


In [14]:
response = ollama.chat(model=model, messages=[
    {
        "role": "user",
        "content": f"--- Document Contents: ---\n{document_contents}\n\n{prompt}"
    }
])
print(response.message.content)
# response = ollama.chat(model=model, messages=[
#     {
#         "role": "system",
#         "content": "You are part of an automated machine to machine interface. Give me only valid JSON outputs. No other text outputs please"
#     },
#     {
#         "role": "user",
#         "content": f"{pre_prompt}\n{document_contents}"
#     }
# ])

Here are the 10 details:

1. **Product Name**: BunkerHill Health Inc.
2. **Company Name**: AI
3. **Sub-specialty**: None specified
4. **Imaging Modality**: Computed Tomography (CT)
5. **Intended Purpose**: To estimate coronary artery calcium burden in patients aged 30 years and above for routine care
6. **Clinical Use**: Routine care
7. **Product Output**: Calcium score, exact Agatston scores including vessel-specific scores
8. **CE Classification**: Not specified
9. **FDA Classification**: Class I (Biologic Health)


In [27]:
response

ChatResponse(model='llama3.2:1b', created_at='2025-07-02T11:51:34.4286268Z', done=True, done_reason='stop', total_duration=27095252400, load_duration=3349548700, prompt_eval_count=2402, prompt_eval_duration=7068631300, eval_count=159, eval_duration=16663847400, message=Message(role='assistant', content='{\n  "AI Product Name": "IB Neuro™ v1.0",\n  "AI Company Name": "Imaging Biometrics, LLC",\n  "Sub-specialty": "Magnetic Resonance Diagnostic Device",\n  "Imaging Modality": "MRI",\n  "Intended Purpose": "Evaluating image intensity variations over time",\n  "Clinical use": "Processing and displaying dynamically acquired MR datasets for clinical applications",\n  "Product output": "Parametric perfusion maps (Relative Cerebral Blood Volume, Cerebral Blood Flow, Mean Transit Time, and Time to Peak)",\n  "CE Classification": "Class II",\n  "FDA Classification": "Substantially Equivalent to predicate devices",\n  "Manufacturer": "Imaging Biometrics, LLC"\n}', thinking=None, images=None, tool

In [43]:
print(response.message.content) # NEW

The text appears to be a regulatory document, likely from the US Food and Drug Administration (FDA). Here's a breakdown of the main points:

**Introduction**

* The device is called BunkerHill Health Inc.'s CAC (Calcium Angiography) device.
* It is intended for use in cardiac care facilities.

**Technical Specifications**

* The device uses Computed Tomography (CT) scans to detect calcium deposits in the heart.
* It has a specific algorithm for detecting calcium, which includes automatic detection and scoring of calcium deposits.
* The device is designed to produce visual representations of detected calcium scores.

**Clinical Significance**

* The device provides information on coronary artery disease risk categories, including CAC (Calcium Angiography) scores in 4 categories: 0-99 Hounsfield Units (HU), 100-399 HU, and two additional categories.
* The device is intended to be used for routine care, with the goal of providing valuable information on cardiac health.

**Performance Data

In [ ]:
print(response.message.content) # OLD

{
  "AI Product Name": "IB Neuro™ v1.0",
  "AI Company Name": "Imaging Biometrics, LLC",
  "Sub-specialty": "Magnetic Resonance Diagnostic Device",
  "Imaging Modality": "MRI",
  "Intended Purpose": "Evaluating image intensity variations over time",
  "Clinical use": "Processing and displaying dynamically acquired MR datasets for clinical applications",
  "Product output": "Parametric perfusion maps (Relative Cerebral Blood Volume, Cerebral Blood Flow, Mean Transit Time, and Time to Peak)",
  "CE Classification": "Class II",
  "FDA Classification": "Substantially Equivalent to predicate devices",
  "Manufacturer": "Imaging Biometrics, LLC"
}


In [29]:
import json

In [37]:
json_response = json.loads(response.message.content)

Code after this is old...

In [ ]:
# pdf_URL = "https://www.accessdata.fda.gov/cdrh_docs/pdf24/K240369.pdf"
# pdf_URL = "https://www.accessdata.fda.gov/cdrh_docs/pdf24/K241480.pdf"
pdf_URL = "https://www.accessdata.fda.gov/cdrh_docs/pdf8/K080762.pdf"
# pdf_URL = "https://www.accessdata.fda.gov/cdrh_docs/pdf16/K161625.pdf" # PdfReadError
response = requests.get(url=pdf_URL)
on_fly_mem_obj = io.BytesIO(response.content)
# pdf_file = PdfReader(on_fly_mem_obj)

All pages with a "Sincerely" are followed by the tech specs i.e., the 510k summary pages - Consistent with most pages

In [36]:
broke = False
for page_num in range(len(pdf_file.pages)):
    if "Sincerely" in pdf_file.pages[page_num].extract_text():
        broke = True
        break
a = str([False, page_num][broke])
a

'4'

In [37]:
len(pdf_file.pages)

6

In [38]:
pdf_file.pages[1].extract_text()

'510(k) Trade Name Manufacturer\nK073167 BrainStat Software Option for GE GE Medical Systems\nSigna MR Scanners 3200 N. Grandview Blvd. W-827\nWaukesha, WI 53188\nK061429 Dynamic MRI Software Option for ESAOTE S.p.A.\nC-Scan, E-ScanXQ and E-Scan Via Siffredi, 58\nOpera 16153 Genova -Italy\nK984224 Perfusion Package for Magnetom Siemens Medical Systems, Inc.\nVision and Symphony MR Systems 186 Wood Avenue South\nIselin, NJ 08830\n5. Device Description:\nlB NeuroTM OsiriX Plugin is software designed to analyze dynamically acquired\ndatasets. Using well-established algorithms, parametric perfusion maps can be\ngenerated such as Relative Cerebral Blood Volume (rCBV), Cerebral Blood Flow\n(CBF), Mean Transit Time (MTT) and Time to Peak (TTP). The strength of our\nsoftware is its ability to extend the productivity of any existing viewer, CAD\nworkstation or PACS via a platform-independent base library that allows for quick\nand seamless integration into existing server and workstation applic

In [32]:
pdf_file.pages[5].extract_text()

'\x01\x01\x01\x01\x01\x01\x01\n\x0b\x1e\x1a\x18\x14\x1c\x0e\x17\x19\x19\x01\x0e\x14\x11\x19\x1d\x16\x01\x0f\x1a\x12\x06\x01\x10\x1c\x11\x13\x17\x1d\x17\x1b\x1a\x11\x19\x01\t\x08\x07\x02\x18\x03 \x0c\n\x0c\x01\x02\x15\x11\x1d\x14\x13\x03\x01\r\x14\x1f\x17\x12\x14\x01\x01 \x01\n\x01\x01-:0\x01/-@13;>5F1?\x01/;>;:->E\x01->@1>E\x01/-8/525/-@5;:\x015:\x01:;:\x08/;:@>-?@\x07\x01 181/@>;/->05;3>-9\x01\x04\x1a\x18\x1c\x05\x013-@10\x07\x01\n/41?@\x01\x18)\x01?@A051?\t\x01)41\x01/;>1\x0121-@A>1?\x01;2\x01@41\x01<>;0A/@\x01->1\x14\x01\n\x01\nJ\x01 \x18-@13;>5F-@5;:\x01;2\x01@41\x01/;>;:->E\x01->@1>E\x01/-8/5A9\x01.A>01:\x015:\x01@41\x012 ;>9\x01;2\x01-\x01>-:31\x01;2\x01\x163-@?@;:\x01\n?/;>1?\t\x01\x18-8/5A9\x01?/;>1\x013>;A<5:3?\x01->1\x010125:10\x01-?\x01;:1\x01;2\x01@41\x012;A>\x01 2;88;C5:3\x01>-:31?\x01;2\x01\x163-@?@;:\x01\nA:5@?\x14\x01\n-\t\x01 \x1c>;A<\x01\x0b\x14\x01\n\x01\x163-@?@;:\x01A:5@?\x01.\t\x01 \x1c>;A<\x01\x0c\x14\x01\x0b\x08\x13\x13\x01\x163-@?@;:\x01A:5@?\x01\n/\t\x01 \x1c>

In [33]:
len(pdf_file.pages[5].extract_text())

3028

In [39]:
pdf_pages = []
for i in range(len(pdf_file.pages)):
    pdf_pages.append(pdf_file.pages[i].extract_text())

In [82]:
print(pdf_pages[0])

EXHIBIT #1
510(k) SUMMARY
This summary of 510(k) safety and effectiveness information is being submitted in
accordance with the requirements of SMDA 1990 and 21 CFR §807.92.
The assigned 510(k) number is:
1. Submitter's Identification: MAY 1 5 2008
Imaging Biometrics, LLC
1035 Katherine Drive
Elm Grove, WI 53122
Contact Person
Michael Schmainda
Phone: (262) 385-1796
Email: mike@imagingbiometrics.com
Date Summary Prepared: 3/4/2008
2. Name of the Device:
Trade Name: IB NeuroTM vl.0
Classification Name: MR Diagnostic Device Accessory: 21 CFR 892.1000
Classification: Class II
Product Code: LNH
Classification Name: Picture Archiving and Communication Systems
21 CFR 892.2050
Classification: Class II
Product Code: LLZ
3. Common or Usual Name:
lB NeuroTM1.0 / PACS device
4. Predicate Device Information:
lB NeuroTM is substantially equivalent to the following legally marketed devices
that are currently cleared by the FDA:
